# RF Image Classification

This code steps through an entire random forest classification. 99.9% of the code is provided, you just need to make sure that you **enter the proper names of your orthomosaic and training data sets**.

This is important: Python doesn't like long file paths. We need to try to not bury this folder many subfolders deep. Maybe place the folder on the Desktop even.

In [ ]:
# Load libraries
import rasterio
import rioxarray as rxr
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('default')
sns.set_palette("husl")

## Read imagery

This chunk simply reads in the UAV orthomosaic we generated in Metashape. Please make sure to load your georeferenced orthomosaic with the projected CRS. Make sure to copy it to the same folder as the notebook file. After reading in the image, the code plots the normal orthomosaic. In the next step, we want to make sure we only have three bands. If there are more, we need to remove the 4th band which is just a transparency layer that we don't need.

Lastly, we plot all three bands separately. You can switch back and forth between the regular orthomosaic, the three bands, and some information on the bands by clicking on the rectangles below the code chunk.

**REPLACE THE NAME OF THE ORTHOMOSAIC WITH YOUR OWN**

In [ ]:
# Read raster with rioxarray
# --> change the name of the tif file to whatever your orthomosaic is called!
img_path = "Ortho3GCPsEPSG6342.tif"  # 3-band RGB UAV orthomosaic
img = rxr.open_rasterio(img_path)

print(f"Image shape: {img.shape}")
print(f"Number of bands: {len(img.band)}")
print(f"CRS: {img.rio.crs}")

# If there are more than 3 bands, keep only the first three (R, G, B)
if len(img.band) > 3:
    print("More than 3 bands detected. Keeping only RGB bands.")
    img = img.isel(band=slice(0, 3))

# Assign band names for clarity
img = img.assign_coords(band=['R', 'G', 'B'])
print(f"\nBand names: {list(img.band.values)}")

# Plot RGB image
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plot RGB composite (stretch to 2nd-98th percentile for better visualization)
rgb_stretched = img.clip(img.quantile(0.02), img.quantile(0.98))
rgb_normalized = (rgb_stretched - rgb_stretched.min()) / (rgb_stretched.max() - rgb_stretched.min())
rgb_plot = np.transpose(rgb_normalized.values, (1, 2, 0))
axes[0].imshow(rgb_plot)
axes[0].set_title('RGB Composite')
axes[0].axis('off')

# Plot individual bands
img.plot.imshow(col='band', col_wrap=3, figsize=(12, 4), 
                cmap='gray', add_colorbar=True)
plt.suptitle('Individual Bands')
plt.tight_layout()
plt.show()

## Read Training Data

Here we just read the training data geopackage into Python that we created in QGIS. Change the file name of the training data set to what yours is called. **REPLACE THE NAME OF THE TRAINING GPKG WITH YOUR OWN**

In [ ]:
# Read the training polygons
# --> enter the name of your training data .gpkg
trn = gpd.read_file("classes.gpkg")

# Make sure the class column is a categorical type
trn['class'] = trn['class'].astype('category')

# Quick look
print("Training data info:")
print(trn.info())
print("\nClass counts:")
print(trn['class'].value_counts())
print("\nFirst few rows:")
print(trn.head())

## Add an ignore zone

THIS IS PROBABLY NOT NECESSARY. SKIP UNTIL YOU THINK YOU MIGHT NEED IT. This chunk adds a layer that will be masked out in the imagery because it contains unnaturally bright colors or other areas which might confuse the indices.

THIS IS NOT NECESSARY IN MOST CASES.

Only go back to this chunk if something seems off in the imagery or calculated indices. Since we probably won't need it, this code won't get executed when the entire script is run.

In [ ]:
# --- Ignore Zone Masking ---
# Uncomment and run this cell only if you need to mask out specific areas

# # Read polygons marking areas to ignore
# ignore = gpd.read_file("ignore_zones_epsg_6342.gpkg")

# # Create a mask from the ignore polygons
# from rasterio import features
# from rasterio.transform import from_bounds

# # Create a mask raster from the ignore polygons
# mask_array = features.rasterize(
#     ignore.geometry,
#     out_shape=img.shape[1:],
#     transform=img.rio.transform(),
#     fill=1,  # areas to keep
#     default_value=0  # areas to ignore
# )

# # Apply mask to the image (inverse=True means "keep everything outside polygons")
# img_masked = img.where(mask_array == 1)
# print("Image masked successfully")

# # Quick check: plot with masked areas
# rgb_masked_plot = np.transpose(img_masked.values, (1, 2, 0))
# rgb_masked_plot = (rgb_masked_plot - np.nanmin(rgb_masked_plot)) / (np.nanmax(rgb_masked_plot) - np.nanmin(rgb_masked_plot))
# plt.figure(figsize=(10, 8))
# plt.imshow(rgb_masked_plot)
# plt.title('RGB with Ignore Zone Masked')
# plt.axis('off')
# plt.show()

## Create simple RGB Indices

We only have three bands, so many of the cool indices available for multispectral data are impossible for us to generate. However, there are some indices that work with just RGB data. We will derive some of them and then also check whether they add much information or not. This chunk creates the indices and then stacks them all together in one variable called stk.

Pay attention to the plot at the end of the code chunk. You want to make sure that there is variability within each of the bands and indices.

In [ ]:
# Normalize to 0-1
# Check if masked image exists, otherwise use the normal image
try:
    img01 = img_masked / 255.0
    print("Using masked image")
except NameError:
    img01 = img / 255.0
    print("Using original image")

# Use a tiny epsilon to prevent division by zero
eps = 1e-10

# Extract individual bands
R = img01.sel(band='R')
G = img01.sel(band='G')
B = img01.sel(band='B')

# Calculate indices
# Visible Atmospherically Resistant Index (VARI)
VARI = (G - R) / (G + R - B + eps)
# Excess Green (ExG)
ExG = 2 * G - R - B
# Normalized Difference Index (NDI, Blue–Red)
NDI = (B - R) / (B + R + eps)
# Green-Red Vegetation Index (GRVI)
GRVI = (G - R) / (G + R + B + eps)

# Clamp extreme outliers so plots look reasonable
VARI = VARI.clip(-1, 1)
ExG = ExG.clip(-1, 1)
NDI = NDI.clip(-1, 1)
GRVI = GRVI.clip(-1, 1)

# Stack all bands and indices together
stk = xr.concat([R, G, B, VARI, ExG, NDI, GRVI], 
                dim='band')
stk = stk.assign_coords(band=['R', 'G', 'B', 'VARI', 'ExG', 'NDI', 'GRVI'])

print(f"Stacked array shape: {stk.shape}")
print(f"Band names: {list(stk.band.values)}")

# Plot all bands and indices
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.ravel()

for i, band_name in enumerate(stk.band.values):
    if i < len(axes):
        band_data = stk.sel(band=band_name)
        im = axes[i].imshow(band_data, cmap='viridis')
        axes[i].set_title(f'{band_name}')
        axes[i].axis('off')
        plt.colorbar(im, ax=axes[i], shrink=0.6)

# Remove the last empty subplot
if len(axes) > len(stk.band.values):
    axes[-1].remove()

plt.tight_layout()
plt.show()

## Extract training data

This chunk takes the training data from QGIS and gives you a summary of the total number of objects in each class and returns them in a table.

Try to get at least 4-5 different classes (e.g., bare earth, water, green vegetation, brown vegetation, different veg that can be identified, etc). More small polygons are generally thought to be superior to fewer large polygons. Make sure to not include any areas that don't belong to the class you are generating a training polygon for.

The lower case n in the table is the number of pixels available in each training data class. If some of the samples are very low compared to others, you might want to consider going back to QGIS and adding a few more polygons of the underrepresented class(es).

In [ ]:
from rasterio import features
from shapely.geometry import mapping

# Function to extract pixel values from raster using vector polygons
def extract_pixels(raster_stack, polygons_gdf):
    extracted_data = []
    
    for idx, row in polygons_gdf.iterrows():
        # Get the geometry
        geom = [mapping(row.geometry)]
        
        # Create a mask for this polygon
        mask = features.rasterize(
            geom,
            out_shape=raster_stack.shape[1:],
            transform=raster_stack.rio.transform(),
            fill=0,
            default_value=1
        )
        
        # Extract pixel values where mask is True
        for band_name in raster_stack.band.values:
            band_data = raster_stack.sel(band=band_name).values
            masked_pixels = band_data[mask == 1]
            
            # Remove NaN values
            masked_pixels = masked_pixels[~np.isnan(masked_pixels)]
            
            for pixel_val in masked_pixels:
                extracted_data.append({
                    'ID': idx,
                    'class': row['class'],
                    'band': band_name,
                    'value': pixel_val
                })
    
    return pd.DataFrame(extracted_data)

# Extract pixel values for training polygons
print("Extracting training data... This may take a few minutes.")
extracted_df = extract_pixels(stk, trn)

# Pivot the data so each row is a pixel and each column is a band
smp = extracted_df.pivot_table(
    index=['ID', 'class'], 
    columns='band', 
    values='value', 
    aggfunc='first'
).reset_index()

# Remove rows with any NaN values
smp = smp.dropna()

print(f"\nExtracted {len(smp)} pixel samples")
print("\nSamples per class:")
print(smp['class'].value_counts().sort_index())

## Check the training data

In the next chunk, we will generate a few plots to assess how well defined the training data classes are. We talked about some of these in the lecture. Note that we normalized the digital numbers further up to 0-1.

In [ ]:
# Check per-class sample sizes
class_counts = smp['class'].value_counts().sort_index()

plt.figure(figsize=(10, 6))
plt.bar(class_counts.index, class_counts.values)
plt.title('Number of Training Samples per Class')
plt.xlabel('Class')
plt.ylabel('Pixel Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Prepare data for long format (similar to pivot_longer in R)
band_columns = ['R', 'G', 'B', 'VARI', 'ExG', 'NDI', 'GRVI']
smp_long = pd.melt(smp, 
                   id_vars=['ID', 'class'], 
                   value_vars=band_columns,
                   var_name='Band', 
                   value_name='Value')

# Histograms per band & class
fig, axes = plt.subplots(len(smp['class'].unique()), len(band_columns), 
                        figsize=(20, 4*len(smp['class'].unique())))

if len(smp['class'].unique()) == 1:
    axes = axes.reshape(1, -1)

for i, class_name in enumerate(sorted(smp['class'].unique())):
    for j, band_name in enumerate(band_columns):
        data_subset = smp_long[(smp_long['class'] == class_name) & 
                              (smp_long['Band'] == band_name)]['Value']
        axes[i, j].hist(data_subset, bins=40, color='gray', alpha=0.7, edgecolor='black')
        axes[i, j].set_title(f'{class_name} - {band_name}')
        axes[i, j].set_xlabel('Digital Number (DN)')
        axes[i, j].set_ylabel('Count')

plt.suptitle('Histograms of Pixel Values by Class and Band', fontsize=16)
plt.tight_layout()
plt.show()

# Overlaid density plots by band
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.ravel()

for i, band_name in enumerate(band_columns):
    if i < len(axes):
        for class_name in sorted(smp['class'].unique()):
            data_subset = smp_long[(smp_long['class'] == class_name) & 
                                  (smp_long['Band'] == band_name)]['Value']
            axes[i].hist(data_subset, bins=30, alpha=0.5, label=class_name, density=True)
        axes[i].set_title(f'{band_name}')
        axes[i].set_xlabel('Digital Number (DN)')
        axes[i].set_ylabel('Density')
        axes[i].legend()

# Remove the last empty subplot
if len(axes) > len(band_columns):
    axes[-1].remove()

plt.suptitle('Spectral Distributions by Class', fontsize=16)
plt.tight_layout()
plt.show()

# Boxplots per band grouped by class
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.ravel()

for i, band_name in enumerate(band_columns):
    if i < len(axes):
        band_data = smp_long[smp_long['Band'] == band_name]
        sns.boxplot(data=band_data, x='class', y='Value', ax=axes[i])
        axes[i].set_title(f'{band_name}')
        axes[i].set_xlabel('Class')
        axes[i].set_ylabel('Digital Number (DN)')
        axes[i].tick_params(axis='x', rotation=45)

# Remove the last empty subplot
if len(axes) > len(band_columns):
    axes[-1].remove()

plt.suptitle('Boxplots of DN Values by Band and Class', fontsize=16)
plt.tight_layout()
plt.show()

## Train Random Forest Model

This chunk runs the actual RF model. For the model, we will use 80% of the available pixels in the training dataset and the remaining 20% in the verification dataset. This means that 80% of the pixels in each class can theoretically be used to improve the model and the model will then be tested on the remaining 20% of the pixels. We will use 301 random forest trees. Ideally you would probably want to choose more, but we will go with a more limited number due to the constraints of the lab computers.

This chunk will both create the model that we will use for the classification and a couple things to evaluate how well the model actually works. The variable importance tells you about, well, how important each of our chosen bands is for the classification. The longer the bar, the more important that particular band or index is for the classification.

In [ ]:
# Set random seed for reproducibility
np.random.seed(7)

# Prepare the data for sklearn
X = smp[band_columns]  # Features (bands and indices)
y = smp['class']       # Target (class labels)

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"Classes: {sorted(y.unique())}")

# Create stratified train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=7, stratify=y
)

print(f"\nTraining set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")
print("\nTraining set class distribution:")
print(y_train.value_counts().sort_index())
print("\nTest set class distribution:")
print(y_test.value_counts().sort_index())

# Train the Random Forest model
print("\nTraining Random Forest model...")
rf_model = RandomForestClassifier(
    n_estimators=301,
    random_state=7,
    n_jobs=-1  # Use all available cores
)

start_time = time.time()
rf_model.fit(X_train, y_train)
training_time = time.time() - start_time

print(f"Training completed in {training_time:.2f} seconds")

# Extract and plot feature importance
feature_importance = pd.DataFrame({
    'Variable': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=True)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Variable'], feature_importance['Importance'])
plt.title('Variable Importance (Random Forest)')
plt.xlabel('Importance')
plt.ylabel('Variable')
plt.tight_layout()
plt.show()

print("\nFeature Importance:")
for var, imp in zip(feature_importance['Variable'], feature_importance['Importance']):
    print(f"{var}: {imp:.4f}")

In [ ]:
# Save the trained model
model_path = "my_trained_rf_model.pkl"
with open(model_path, 'wb') as f:
    pickle.dump(rf_model, f)

print(f"Model saved to: {model_path}")

In [ ]:
# Load the model back
model_path = "my_trained_rf_model.pkl"
with open(model_path, 'rb') as f:
    rf_model = pickle.load(f)

print("Model loaded successfully")

## Accuracy Assessment

This chunk uses the test data (pixels) that we set aside earlier to use in a test run. We apply the model to the test dataset, where we know exactly what the correct classification should be. This allows us to estimate the general accuracy of the model down to the class level (i.e., do predictions work better for some classes than for others).

This is where you tinker with the model. Well, not the actual model, but with the training data. If you see classes that aren't predicted well, go back to QGIS and check your training polygons. If classes are too similar, try to find training polygons that are more different. Or combine classes. Use your best knowledge of training data to refine the model outcome!

The confusion matrix is really useful for checking how well the model predicts the individual classes. This is where the remaining 20% of the training data set come into play. On the Y axis, we have the actual class, and on the X axis we have the predicted class. Each row adds up to 100%. You want to see really high percentages for a given class and the prediction of the given class. If the training data are ambiguous for a class you will probably see that there are high percentages for each given class that are predicted incorrectly. If the majority of a class is misclassified, either consider dropping the class or creating, if possible, better training polygons for that class.

The table gives you some additional classification metrics.

In [ ]:
# Predict on the test set
y_pred = rf_model.predict(X_test)

# Calculate overall accuracy
overall_accuracy = accuracy_score(y_test, y_pred)
print(f"Overall Accuracy: {overall_accuracy:.3f}")

# Create confusion matrix
cm = confusion_matrix(y_test, y_pred)
class_names = sorted(y_test.unique())

# Normalize confusion matrix by row (actual class) to get percentages
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, 
            annot=True, 
            fmt='.1f', 
            cmap='Blues',
            xticklabels=class_names, 
            yticklabels=class_names,
            cbar_kws={'label': 'Percentage'})
plt.title('Confusion Matrix (Normalized by Row - %)')
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')
plt.tight_layout()
plt.show()

# Print detailed classification report
print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred))

# Create a DataFrame with per-class metrics
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1, support = precision_recall_fscore_support(y_test, y_pred)

metrics_df = pd.DataFrame({
    'Class': class_names,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1,
    'Support': support
})

print("\nPer-class Metrics:")
print(metrics_df.round(3))

## Predict the raster

In this chunk, we apply the classification to the full image.

THIS STEP IS GOING TO TAKE A WHILE! (20-30 minutes would be my guess; it took about 40 minutes on an 8-year old laptop, and 13 minutes on a very fast desktop computer) Note that the first code line in this chunk sets processing parameters. There is a possibility that the lab computers don't allow us to store data on the C drive. If that is the case, we may need to change it to a temp folder in our OneDrive. There is also a small chance it might crash your computer... Or just take way too long. If that is the case, I will run the classification for you on my office computer (after the lab) and send you the results (hopefully we can avoid this, though). If you need to do this, please keep all the necessary files in the same folder, compress the folder (right-click, compress as zip) and then email it to me. I will send you the completed classification back.

In [ ]:
# Predict classification for every pixel
print("Starting full image classification...")
print(f"Image dimensions: {stk.shape[1]} x {stk.shape[2]} pixels")
print(f"Total pixels to classify: {stk.shape[1] * stk.shape[2]:,}")

start_time = time.time()

# Reshape the stack data for prediction
# Convert from (bands, height, width) to (pixels, bands)
stk_array = stk.values
n_bands, n_rows, n_cols = stk_array.shape

# Reshape to (n_pixels, n_bands)
pixels_array = stk_array.reshape(n_bands, -1).T

# Remove pixels with NaN values
valid_mask = ~np.isnan(pixels_array).any(axis=1)
valid_pixels = pixels_array[valid_mask]

print(f"Valid pixels for classification: {len(valid_pixels):,}")

# Predict on valid pixels
if len(valid_pixels) > 0:
    print("Applying Random Forest model...")
    predictions = rf_model.predict(valid_pixels)
    
    # Create full prediction array with NaN for invalid pixels
    full_predictions = np.full(n_rows * n_cols, np.nan)
    full_predictions[valid_mask] = predictions
    
    # Reshape back to image dimensions
    class_array = full_predictions.reshape(n_rows, n_cols)
    
    # Create xarray DataArray for the classification result
    class_raster = xr.DataArray(
        class_array,
        coords=[stk.y, stk.x],
        dims=['y', 'x'],
        attrs=stk.attrs
    )
    
    # Save the classification result
    class_raster.rio.write_crs(stk.rio.crs, inplace=True)
    class_raster.rio.to_raster("classified_map_python.tif")
    
    classification_time = time.time() - start_time
    print(f"Classification completed in {classification_time:.2f} seconds")
    print("Classification saved as 'classified_map_python.tif'")
    
else:
    print("No valid pixels found for classification!")

# Plot the classification result
if 'class_raster' in locals():
    plt.figure(figsize=(12, 10))
    
    # Create a custom colormap for the classes
    unique_classes = sorted([c for c in np.unique(class_raster.values) if not np.isnan(c)])
    n_classes = len(unique_classes)
    
    # Plot the classification
    im = plt.imshow(class_raster.values, cmap='Set3', vmin=0, vmax=n_classes-1)
    
    # Add colorbar with class labels
    cbar = plt.colorbar(im, shrink=0.6)
    cbar.set_label('Class')
    
    plt.title('Random Forest Classification of UAV RGB Imagery')
    plt.xlabel('Easting')
    plt.ylabel('Northing')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()
    
    # Print classification summary
    print("\nClassification Summary:")
    unique_values, counts = np.unique(class_raster.values[~np.isnan(class_raster.values)], return_counts=True)
    for val, count in zip(unique_values, counts):
        print(f"Class {int(val)}: {count:,} pixels ({count/np.sum(counts)*100:.1f}%)")

So, now you are done with an image classification! Having the orthomosaic and classification in Python is good, but you can better evaluate the results if you copy both of the tifs into QGIS. Just note that you won't see the class names in QGIS, they will be labeled 1 through x, with x being your number of classes.

## QUESTIONS

**Q1 (5 pts):** Describe the process for supervised classification. Within this, briefly explain the random forest approach.

*Answer:*

**Q2 (5 pts):** Discuss how well your classes are defined (what classes did you use and why?). Use the spectral plots for this answer.

*Answer:*

**Q3 (5 pts):** Connected to Q2, how well does the model perform? Assess it using the tables and graphs that you generated (Variable Importance and Confusion Matrix).

*Answer:*

**Q4 (4 pts):** Discuss your classification results in general outside of the training data evaluation. What were issues/difficulties during the classification process? How could you improve the classification?

*Answer:*

**Q5 (5 pts):** In addition to answering the questions, include the resulting notebook file. That should ideally have all the plots and figures.

*Answer:*